<a href="https://colab.research.google.com/github/pko89403/Recsys_test/blob/master/item2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pwd

/content


In [25]:
!ls /content/

drive  sample_data


In [0]:
import pandas as pd

RATINGS_PATH = "/content/drive/My Drive/data/ml-1m/ratings.dat"
MOVIES_PATH = "/content/drive/My Drive/data/ml-1m/movies.dat"

In [0]:
ratings = pd.read_csv(RATINGS_PATH, header=None, sep="::", engine="python")
movies = pd.read_csv(MOVIES_PATH, header=None, sep="::", engine="python")

In [0]:
ratings.columns = ["uuid", "iid", "rating", "timestamp"]
movies.columns = ["iid", "movie_name", "genre"]

In [29]:
ratings.head()
movies.head()

,iid,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [0]:
iid_to_movie_name = dict( zip(movies.iid.tolist(), movies.movie_name.tolist()))
movie_name_to_iid = { v: k for k, v in iid_to_movie_name.items()}

In [31]:
iid_to_movie_name

{1: 'Toy Story (1995)',
 2: 'Jumanji (1995)',
 3: 'Grumpier Old Men (1995)',
 4: 'Waiting to Exhale (1995)',
 5: 'Father of the Bride Part II (1995)',
 6: 'Heat (1995)',
 7: 'Sabrina (1995)',
 8: 'Tom and Huck (1995)',
 9: 'Sudden Death (1995)',
 10: 'GoldenEye (1995)',
 11: 'American President, The (1995)',
 12: 'Dracula: Dead and Loving It (1995)',
 13: 'Balto (1995)',
 14: 'Nixon (1995)',
 15: 'Cutthroat Island (1995)',
 16: 'Casino (1995)',
 17: 'Sense and Sensibility (1995)',
 18: 'Four Rooms (1995)',
 19: 'Ace Ventura: When Nature Calls (1995)',
 20: 'Money Train (1995)',
 21: 'Get Shorty (1995)',
 22: 'Copycat (1995)',
 23: 'Assassins (1995)',
 24: 'Powder (1995)',
 25: 'Leaving Las Vegas (1995)',
 26: 'Othello (1995)',
 27: 'Now and Then (1995)',
 28: 'Persuasion (1995)',
 29: 'City of Lost Children, The (1995)',
 30: 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 31: 'Dangerous Minds (1995)',
 32: 'Twelve Monkeys (1995)',
 33: 'Wings of Courage (1995)',
 34: 'Babe (1

In [32]:
movie_name_to_iid

{'Toy Story (1995)': 1,
 'Jumanji (1995)': 2,
 'Grumpier Old Men (1995)': 3,
 'Waiting to Exhale (1995)': 4,
 'Father of the Bride Part II (1995)': 5,
 'Heat (1995)': 6,
 'Sabrina (1995)': 7,
 'Tom and Huck (1995)': 8,
 'Sudden Death (1995)': 9,
 'GoldenEye (1995)': 10,
 'American President, The (1995)': 11,
 'Dracula: Dead and Loving It (1995)': 12,
 'Balto (1995)': 13,
 'Nixon (1995)': 14,
 'Cutthroat Island (1995)': 15,
 'Casino (1995)': 16,
 'Sense and Sensibility (1995)': 17,
 'Four Rooms (1995)': 18,
 'Ace Ventura: When Nature Calls (1995)': 19,
 'Money Train (1995)': 20,
 'Get Shorty (1995)': 21,
 'Copycat (1995)': 22,
 'Assassins (1995)': 23,
 'Powder (1995)': 24,
 'Leaving Las Vegas (1995)': 25,
 'Othello (1995)': 26,
 'Now and Then (1995)': 27,
 'Persuasion (1995)': 28,
 'City of Lost Children, The (1995)': 29,
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 30,
 'Dangerous Minds (1995)': 31,
 'Twelve Monkeys (1995)': 32,
 'Wings of Courage (1995)': 33,
 'Babe (1995)

Total Items는 중요하다. Total Items가 Word2Vec에서의 Vocab Size와 같다.

In [33]:
only_user = ratings.uuid.unique().tolist()
total_items = len(movie_name_to_iid)
print(f"Total Users : {len(only_user)}, Total Items : {total_items}")


Total Users : 6040, Total Items : 3883


유저 하나당 평점 매긴 영화들의 리스트를 만든다.    
왜냐면 그걸로 Item2Vec의 데이터로 사용할 것이기 때문에

In [0]:
skip_item_dataset = [ratings[ratings.uuid==user]['iid'].tolist() for user in only_user]

In [0]:
with open('/content/drive/My Drive/data/ml-1m/skip_item.dat', 'w') as f:
  for line in skip_item_dataset:
    f.write(str(line)+'\n')

In [0]:
def proc_item_freq(skip_item_dataset):
  total_items_with_dup = 0
  item_frequency = dict()
  for rate_hist in skip_item_dataset:
    for item in rate_hist:
      total_items_with_dup += 1
      try:
        item_frequency[item] += 1
      except:
        item_frequency[item] = 1
  return item_frequency, total_items_with_dup

In [0]:
item_freq, total_items_with_dup = proc_item_freq(skip_item_dataset)

In [0]:
import numpy as np 

def create_sample_table(item_frequency):
  sample_table_size = 1e6

  freq_dist = [ f ** 0.75 for f in item_frequency.values()]
  prob_dist = np.array(freq_dist) / sum(freq_dist)
  count_dist = np.round( prob_dist * sample_table_size)

  sample_table = list()
  for iid, cnt in enumerate(count_dist):
    sample_table += ( [iid] * int(cnt) )
  
  return sample_table

In [0]:
sample_table = create_sample_table(item_freq) 

In [0]:
def get_positive_pair(dataset, window_size, batch_size):
  pairs = list()
  
  while(len(pairs) < batch_size):
    open_file = dataset
    rate_hist = list()
    
    for line in open_file:
      rate_hist = line 

    for pos, u in enumerate(rate_hist):
      for v in rate_hist[max(pos-window_size,0): min(pos+window_size,len(rate_hist))]:
        if u == v:
          continue
        pairs.append((u, v))
    
  
  positive_pairs = random.sample(pairs,batch_size)

  return positive_pairs

import random 

def negative_samples(sample_table, k=5):
  return np.random.choice(sample_table, size=(1024,k))




In [0]:
positive_pairs = get_positive_pair(dataset=skip_item_dataset, window_size=3, batch_size=1024)

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F 

In [0]:
class SGNS(nn.Module):
  def __init__(self, total_items, embedding_dims):
    super(SGNS, self).__init__()
    self.u_embedding = nn.Embedding(total_items, embedding_dims)
    self.v_embedding = nn.Embedding(total_items, embedding_dims)
  
  def forward(self, u_idx, v_pos_idx, v_neg_idx):
    u_emb_vec = self.u_embedding(u_idx)
    v_pos_emb_vec = self.v_embedding(v_pos_idx)


    print(v_pos_emb_vec.shape)
    pos_pred = torch.mul(u_emb_vec, v_pos_emb_vec)
    print(pos_pred.shape)
    pos_pred = pos_pred.squeeze()
    print(pos_pred.shape)
    pos_pred = torch.sum(pos_pred, dim=1)
    print(pos_pred.shape)
    pos_pred = F.logsigmoid(pos_pred)
    print(pos_pred.shape)

    v_neg_emb_vec = self.v_embedding(v_neg_idx)
    print('v_neg_emb_vec - ', v_neg_emb_vec.shape)
    print('u_emb_vec - ', u_emb_vec.shape)
    u_emb_vec = u_emb_vec.unsqueeze(2)
    print('u_emb_vec unsqz2- ', u_emb_vec.shape)
    neg_pred = torch.bmm(v_neg_emb_vec, u_emb_vec)
    print('neg_pred - ', neg_pred.shape)
    neg_pred = neg_pred.squeeze()
    print('neg_pred sqz - ', neg_pred.shape)
    neg_pred = F.logsigmoid(-1 * neg_pred)

    return -1 * ( torch.sum(pos_pred) + torch.sum(neg_pred))


In [0]:
item2vec = SGNS(total_items=total_items, embedding_dims=100).cuda()
optimizer = torch.optim.SGD(item2vec.parameters(), lr = 1e-3)
epoch = 10

In [97]:
for i in range(epoch):
  pairs = get_positive_pair(dataset=skip_item_dataset, window_size=3, batch_size=1024)

  u_idx = Variable(torch.Tensor([pair[0] for pair in pairs]).long()).cuda()
  v_idx = Variable(torch.Tensor([pair[1] for pair in pairs]).long()).cuda()
  v_neg = Variable(torch.Tensor(negative_samples(sample_table=sample_table, k=5)).long()).cuda()

  print('u_idx : ', u_idx, '\n')

  loss = item2vec(u_idx, v_idx, v_neg)
  #optimizer.zero_grad()
  #loss.backward()
  #optimizer.step()

u_idx :  tensor([ 919,   29, 1285,  ..., 2357,   25,  589], device='cuda:0') 

torch.Size([1024, 100])
torch.Size([1024, 100])
torch.Size([1024, 100])
torch.Size([1024])
torch.Size([1024])
v_neg_emb_vec -  torch.Size([1024, 5, 100])
u_emb_vec -  torch.Size([1024, 100])
u_emb_vec unsqz2-  torch.Size([1024, 100, 1])
neg_pred -  torch.Size([1024, 5, 1])
neg_pred sqz -  torch.Size([1024, 5])
u_idx :  tensor([ 232, 1103, 2716,  ..., 3361, 2391, 2858], device='cuda:0') 

torch.Size([1024, 100])
torch.Size([1024, 100])
torch.Size([1024, 100])
torch.Size([1024])
torch.Size([1024])
v_neg_emb_vec -  torch.Size([1024, 5, 100])
u_emb_vec -  torch.Size([1024, 100])
u_emb_vec unsqz2-  torch.Size([1024, 100, 1])
neg_pred -  torch.Size([1024, 5, 1])
neg_pred sqz -  torch.Size([1024, 5])
u_idx :  tensor([1294, 2410,  904,  ..., 2750, 1296, 1358], device='cuda:0') 

torch.Size([1024, 100])
torch.Size([1024, 100])
torch.Size([1024, 100])
torch.Size([1024])
torch.Size([1024])
v_neg_emb_vec -  torch.Size([